In [1]:
from numba import cuda
import numpy as np

## Data cleaning

In [5]:
with open("input.txt") as fh:
    inputs = [row.split(":") for row in fh.readlines()]
    inputs = [(*rule.split(" "), password.strip()) for rule, password in inputs]
    inputs = [(*boundaries.split("-"), letter, password) for boundaries, letter, password in inputs]
    inputs = [(int(lower), int(upper), letter, password) for lower, upper, letter, password in inputs]
    
    lowers = [x[0] for x in inputs]
    uppers = [x[1] for x in inputs]
    letters = [ord(x[2]) for x in inputs]
    passwords = [[ord(char) for char in x[3]] for x in inputs]
    lengths = [len(password) for password in passwords]
    
    max_length = max(lengths)
    for i in range(len(passwords)):
        if len(passwords[i]) < max_length:
            passwords[i].extend([0] * (max_length - len(passwords[i])))
            
    dtype = [
        ('lower', '<i4'),
        ('upper', '<i4'),
        ('letter', '<i4'),
        ('length', '<i4'),
        ('password', '<i4', max(lengths))
    ]

    entries = np.array(list(zip(lowers, uppers, letters, lengths, passwords)),
                       dtype=dtype)

In [8]:
entries[:10]

array([( 1,  9, 120, 14, [120, 119, 106, 103, 120, 116, 109, 114, 122, 120, 122, 109, 107, 120,   0,   0,   0,   0,   0,   0]),
       ( 4,  6, 114,  7, [114, 114, 114, 107, 114, 103, 114,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]),
       ( 4,  5, 118,  7, [118, 118, 102, 118, 118, 118, 110,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]),
       ( 5, 16, 109, 16, [112, 120, 109, 114, 116, 109,  98, 109, 113, 109,  99, 108, 100, 109, 109, 109,   0,   0,   0,   0]),
       (15, 16, 115, 20, [ 98, 115, 115, 104, 115, 115, 122, 115, 108, 115, 115, 115, 115, 115, 108, 113, 100, 115, 115, 115]),
       (10, 12, 103, 13, [103, 103, 103, 103, 103, 103, 103, 103, 103, 122, 103, 118, 103,   0,   0,   0,   0,   0,   0,   0]),
       ( 2,  7, 110,  7, [100, 110, 116, 110, 114, 110, 103,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]),
       (11, 14, 106, 16, [120, 114, 106, 102, 108,  98, 109, 106, 115, 122, 122, 106,  98, 106, 106, 104

In [9]:
answer = np.zeros(1)
answer

array([0.])

## Part 1

In [14]:
@cuda.jit
def is_password_valid(entries, answer):
    pos = cuda.grid(1)
    if pos < len(entries):     
        entry = entries[pos]
        lower = entry["lower"]
        upper = entry["upper"]
        letter = entry["letter"]
        password = entry["password"]
        length = entry["length"]
        
        count = 0
        for i in range(length):
            if password[i] == letter:
                count += 1 
        
        if count >= lower and count <= upper:
            cuda.atomic.add(answer, 0, 1)

In [15]:
threadsperblock = 128
blockspergrid = (len(entries) + (threadsperblock - 1)) // threadsperblock

In [16]:
is_password_valid[blockspergrid, threadsperblock](entries, answer)
answer

array([640.])

## Part 2

In [18]:
answer = np.zeros(1)

@cuda.jit
def is_password_valid_2(entries, answer):
    pos = cuda.grid(1)
    if pos < len(entries):     
        entry = entries[pos]
        lower = entry["lower"]
        upper = entry["upper"]
        letter = entry["letter"]
        password = entry["password"]
        length = entry["length"]
        
        count = 0
        if password[lower-1] == letter:
            count += 1
        if password[upper-1] == letter:
            count += 1
        
        if count == 1:
            cuda.atomic.add(answer, 0, 1)

is_password_valid_2[blockspergrid, threadsperblock](entries, answer)
answer

array([472.])